In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pandas as pd
from scipy.signal import savgol_filter
#from astropy.io import fits
#from astropy.coordinates import SkyCoord
#import astropy.units as u
import os.path
import os
import glob
from wotan import flatten

#rom astropy.stats import BoxLeastSquares
from astropy.timeseries import BoxLeastSquares

%matplotlib inline
plt.rcParams.update({'font.size': 15})

In [ ]:
#print((35./58)**2*(0.3**4))
#print(365*np.sqrt(0.06**3/0.3))

In [ ]:
filename = "PC_VKostov_RyanLCs_042519.txt"#"targets_TB.txt"#
tic_ = pd.read_csv(filename, header = 0, sep = ' ')
#filename = "PC_EKruse_RyanLCs_042919.txt"
#tic_ = pd.read_csv(filename, header = None, sep = ' ')
tic_ = np.asarray(tic_)
tic = tic_[:,0].flatten()
tic_bak = tic
period = tic_[:,1].flatten()
depth = tic_[:,2].flatten()
comment = tic_[:,3].flatten()
rs = tic_[:,4].flatten()
#rp = (tic_[:,2]*tic_[:,3])
#tic = np.hstack((25065206, tic))
#for ii in range(len(tic)):
#for fn_ in glob.glob(fp + str(tic[ii]) + "*.lc"):
#print(fn_)
#tar -cvf allfiles.tar -T mylist.txt

In [ ]:
def get_data_(tic, sector):
    file_path_ = "/Users/vkostov/Desktop/Ideas_etc/TESS/tesseract/ryan_LCs/data/"

    for fn_ in glob.glob(file_path_ + str(tic) + "_sector0" + str(sector) + "*.lc"):
        data = pd.read_csv(fn_, sep = ' ')
        data = np.array(data)
        time = data[:,0]
        mag_raw = data[:,1]
        flux = 10**(-0.4*data[:,1])
        flux_nrm = flux/np.nanmean(flux)
        
        return time, flux

In [ ]:
# DETREND DATA

def detrend_data_(time, flux, do_plot_, sector):
    x = time
    y = flux/np.nanmedian(flux)
    mu = np.median(y)
    y = (y / mu - 1)# * 1e3

# Identify outliers
    m = np.ones(len(y), dtype=bool)
    
    for i in range(10):
        y_prime = np.interp(x, x[m], y[m])
        smooth = savgol_filter(y_prime, 121, polyorder=3)
        resid = y - smooth
        sigma = np.sqrt(np.mean(resid**2))
        m0 = np.abs(resid) < 3*sigma
        if m.sum() == m0.sum():
            m = m0
            break
        m = m0

# Only discard positive outliers
    if sector < 3:
        m = (resid < 3*sigma) & (resid > -5*sigma)
    else:
        m = (resid < 2*sigma) & (resid > -3*sigma)
# REMOVE DATA AROUND DATA GAPS/DISCONTINUITIES!!!!
    if sector < 3:
        m[(x - np.min(x) >= 12.25) & (x - np.min(x) <= 15.3)] = False
    elif sector == 3:
        m[(x - np.min(x) >= 11.5) & (x - np.min(x) <= 14.5)] = False
        m[(x - np.min(x) <= 2.25)] = False
        m[(x - np.min(x) > 26.5)] = False 
    elif sector == 4:
        m[(x - np.min(x) >= 7.) & (x - np.min(x) <= 15.)] = False
        m[(x - np.min(x) >= 25.)] = False
    
    m = (resid < 5*sigma) & (resid > -15*sigma)
#
# Shift the data so that it starts at t=0. This tends to make the fit
# better behaved since t0 covaries with period.
#    print(min(x), min(x[m]))
    x_ref = np.min(x[m]) if sector != 3 else np.min(x)
#    x -= x_ref

# Make sure that the data type is consistent
    x_bak_, y_bak_ = x, y
    x = np.ascontiguousarray(x[m], dtype=np.float64)
    y = np.ascontiguousarray(y[m], dtype=np.float64)
    smooth = np.ascontiguousarray(smooth[m], dtype=np.float64)

    x_detrend_ = x
    y_detrend_ = y - smooth
    
#    do_plot_= 'y'
    if do_plot_ == 'y':
        plt.figure(figsize=(20,5))
    
# Plot the data
#        plt.subplot(211)
        plt.plot(x_bak_+0*x_ref, y_bak_, "k", label="raw data")
        plt.plot(x+0*x_ref, smooth, lw = 3., label = 'sav-gol filter')
        plt.plot(x_bak_[~m]+0*x_ref, y_bak_[~m], "xr", label="outliers")
        plt.legend(fontsize=12)
#        plt.xlim(x_bak_.min(), x_bak_.max())
        plt.xlabel("time")
        plt.ylabel("flux")
#        plt.ylim(-25., 25.)
        
#        plt.subplot(212)
#        plt.plot(x_detrend_, y_detrend_, "k", label="data")
#        plt.legend(fontsize=12)
#        plt.xlim(x_detrend_.min(), x_detrend_.max())
#        plt.xlabel("time")
#        plt.ylabel("flux")
        plt.show()
    
    return x_detrend_, y_detrend_, x_ref, x_bak_, y_bak_, smooth, m

In [ ]:
#RUN BLS
def run_bls_(x, y, tic, do_plot_, min_period, max_period_):
        
    x_ref = min(x)
    x -= min(x)
    
    period_grid = np.exp(np.linspace(np.log(min_period), np.log(max_period_), 50000))
    durations_ = 0.1+0.05*np.linspace(0,3,4)#0.02+0.02*np.linspace(0,11,12)#0.02+0.02*np.linspace(0,11,12)
    bls_results = []
    periods = []
    t0s = []
    depths = []
    durations = []

    # Compute the periodogram for each planet by iteratively masking out
    # transits from the higher signal to noise planets. Here we're assuming
    # that we know that there are exactly two planets.
    for i in range(1):
        bls = BoxLeastSquares(x, y)
        bls_power = bls.power(period_grid, durations_, oversample=20)
        bls_results.append(bls_power)
    
    # Save the highest peak as the planet candidate
        index = np.argmax(bls_power.power)
        periods.append(bls_power.period[index])
        t0s.append(bls_power.transit_time[index])
        depths.append(bls_power.depth[index])
        durations.append(bls_power.duration[index])
        
    # Mask the data points that are in transit for this candidate
        m = np.zeros(len(x), dtype=bool)
        m |= bls.transit_mask(x, periods[-1], 0.5, t0s[-1])
        
    # PLOT RESULTS   

    if do_plot_ == 'y':

        fig, axes = plt.subplots(len(bls_results), 2, figsize=(20, 10))
        
        for i in range(len(bls_results)):
    # Plot the lightcurve
            plt.subplot(2,3,(1,3))
            plt.plot(x+x_ref, y, 'k-')
            plt.title('detrended TESS lightcurve')
            plt.ylabel('[ppt]')
            plt.xlim((x + x_ref).min(), (x + x_ref).max())
    # Plot the periodogram 
            plt.subplot(234)
            plt.plot(np.log10(bls_results[i].period), bls_results[i].power, "k")
            plt.title('BLS periodogram')
    # Plot the folded transit (folded on the full period)            
            plt.subplot(235)
            p = periods[i]
            x_fold = (x - t0s[i] + 0.5*p) % p - 0.5*p
            m = np.abs(x_fold) < p
            plt.plot(x_fold[m], y[m], ".k")  
            inds = np.argsort(x_fold[m])
    # Overplot the box           
            plt.plot(x_fold[m][inds], bls.model(x, p, durations[i], t0s[i])[m][inds], 'C1',lw = 2)
            plt.ylim(-10.*depths[0],5*depths[0])
            plt.xlabel('Phase [days]')
#
#
    # Plot the folded transit (folded on 1-day)   
            plt.subplot(236)
            p = periods[i]
            x_fold = (x - t0s[i] + 0.5*p) % p - 0.5*p
            m = np.abs(x_fold) < 1.0 
            plt.plot(x_fold[m], y[m], ".k")  
            inds = np.argsort(x_fold[m])
    # Overplot the box           
            plt.plot(x_fold[m][inds], bls.model(x, p, durations[i], t0s[i])[m][inds], 'C1',lw = 2)
    # Overplot the phase binned light curve
            bins = np.linspace(-1., 1., 32)#
            denom, _ = np.histogram(x_fold, bins)
            num, _ = np.histogram(x_fold, bins, weights=y)
            denom[num == 0] = 1.0
#            plt.plot(0.5*(bins[1:] + bins[:-1]), num / denom, color="c", lw = 1)
            plt.title('TIC ' + str(tic) + "; P =" + str(np.round(p,2)) + "; to =" + str(round(x_ref+t0s[0],2)))
            plt.xlabel('Phase [days]')
            plt.ylim(-10.*depths[0],5*depths[0])
        
            plt.show()
#        plt.savefig("/Users/vkostov/Desktop/Ideas_etc/TESS/tesseract/ryan_LCs/pngs/" + str(tic) + ".png")
#        plt.close()

    return bls_power

In [ ]:
short_period_tics_ = np.asarray([33864807, 140997881, 55369125, 142016953, 167808509, 177312861, 287351569, 307368125])
long_period_tics_ = np.asarray([30314203,220407125])

In [ ]:
sector = [1,2,3,4]
min_period = 1.

for ii in range(1):#len(tic)):
    
    time, flux_raw = 0., 0.
    time_detrend_, flux_detrend_ = 0., 0.
    
    tic_ = tic[ii]##231081369#364395234#220407125#tic[ii]#382629173#41459998#33863167#
    
    idx_short_, = np.where(short_period_tics_ == tic_)
    idx_long_, = np.where(long_period_tics_ == tic_)
    
    if len(idx_short_ > 0):
        min_period = 0.5
    elif len(idx_long_ > 0):
        min_period = 2.
    else:
        min_period = 1.
    
#    if (tic_ == 33864807) or (tic == 140997881) or (tic_ == 55369125) or (tic_ == 142016953) or \
#    (tic_ == 167808509) or (tic_ == 177312861) or (tic_ == 287351569) or (tic_ == 307368125):
#        min_period = 0.5
#    elif tic_ == 30314203:
#        min_period = 2.
#    else:
#        min_period = 1.
               
    try:
        for jj in range(len(sector)):
            try:
                do_plot_ = 'n'
                if sector[jj] == 1:
                    time, flux_raw = get_data_(tic_, sector[jj])
                    time_detrend_, flux_detrend_, t0, x_bak_, y_bak_, smooth, m = \
                    detrend_data_(time, flux_raw, do_plot_, sector[jj])                
                    time_detrend_ = time_detrend_ + 0.*t0
                else:
                    time_, flux_ = get_data_(tic_, sector[jj])
                    time_tmp_, flux_tmp_, t0_, x_tmp_, y_tmp_, smooth, m_tmp = \
                    detrend_data_(time_, flux_, do_plot_, sector[jj])
            
                    time_detrend_ = np.hstack((time_detrend_,time_tmp_ + 0.*t0_))
                    flux_detrend_ = np.hstack((flux_detrend_, flux_tmp_))
            except:
                do_ = 'nothing'
#                print(tic_, 'no Sector', str(sector[jj]), 'data')  
            
    
#        print(time_detrend_[0:10])
#    if (len(sector) == 1) & (sector != 1):
        time_detrend_, flux_detrend_ = time_detrend_[1:], flux_detrend_[1:]
        t_ref = time_detrend_[0]
    
        do_plot_ = 'y'
        max_period = int((0.8*(max(time_detrend_) - min(time_detrend_))))
        bls_results = run_bls_(time_detrend_, flux_detrend_, tic_, do_plot_, min_period, max_period)
#        print(tic_, '...DONE (', str(ii), ')')
        dur_ = 24*bls_results.duration[np.argmax(bls_results.power)]
#        print('clip = pp_tess.runOneDv("ryan", 1,',tic_, ',1,',
#              np.round(bls_results.period[np.argmax(bls_results.power)],2),',',
#              np.round(bls_results.transit_time[np.argmax(bls_results.power)],4),',',
#              int(1e3*np.round(bls_results.depth[np.argmax(bls_results.power)],0)), ',',
#              "%.2f" % dur_,')')        
        print(tic_,
              np.round(bls_results.period[np.argmax(bls_results.power)],2),
              np.round(t_ref + bls_results.transit_time[np.argmax(bls_results.power)],4),
              "%.2f" % dur_,
              int(1e3*np.round(bls_results.depth[np.argmax(bls_results.power)],0)))
    
        time_bak, flux_bak = time_detrend_, flux_detrend_
        del time, flux_raw, time_detrend_, flux_detrend_
#        print('################################')
#        print('################################')
    except:
        print(tic_, '...NO DATA')

In [ ]:
xxxx

In [ ]:
time, flux = get_data_(364395234, 1)

flatten_lc, trend_lc = flatten(time, flux, window_length=0.5, method='biweight', return_trend=True)

plt.figure(figsize=(20,5))
plt.plot(time, flux, 'k')
#plt.plot(time, 1.-flatten_lc, 'r')

In [ ]:
fig = plt.subplots(figsize=(20, 5))
plt.plot(time_bak+1.*t_ref, flux_bak, 'ko-')
#plt.xlim(1385,1410)
plt.xlim(t_ref+140.,t_ref+160.)
plt.ylim(-20.,10)

#print(t_ref + [5., 62., 76.])

In [ ]:
#fig = plt.subplots(figsize=(20, 5))
#plt.plot(time_bak, flux_bak, 'k-')
#plt.xlim(3.+18,10.+18)

In [ ]:
xxxx

In [ ]:
time, flux = time_bak - 0.*min(time_bak), flux_bak

idx = np.argmax(bls_results.power)
per = bls_results.period[idx]
dur = bls_results.duration[idx]
t0 = bls_results.transit_time[idx]

kk_ = int(np.floor(max(time)/per))
print(kk_)

fig = plt.subplots(figsize=(5, 15))

#mm = [0,1,2,3,15,16,17,27,28,30,31]

if kk_ > 5:
    kk_ = 10

mm = np.linspace(0,kk_-1,kk_)#[0,1,2,3,15,16,17,27,28,30,31]
#mm = [0,1,2,3]

for ii in range(len(mm)):
#    print(ii)
    plt.subplot(len(mm),1,ii+1)
    plt.plot(time, flux, 'k.')
#    plt.xlim(min(time-time[0]), max(time-time[0]))
    plt.xlim(-1. + t0 + ii*per, 1. + t0 + ii*per)
    plt.plot([t0+ii*per,t0+ii*per], [-5, 5],'r--')
#    print(-1.5 + t0 + ii*per, 1.5 + t0 + ii*per, t0+ii*per)
    plt.ylim(-10., 10.)
    
    if ii == 0:
        plt.title('TIC ' + str(tic_), fontsize = 18)
    
#plt.subplot(2,1,2)
#jj = 2
#plt.plot(time, flux, 'k.')
#plt.xlim(min(time-time[0]), max(time-time[0]))
#plt.xlim(-1 + bls_t0s[-1] + jj*bls_periods[-1], 1 + bls_t0s[-1] + jj*bls_periods[-1])
#plt.plot([bls_t0s[-1]+jj*bls_periods[-1],bls_t0s[-1]+ jj*bls_periods[-1]], [-5, 5],'r--')

#plt.savefig('planet_X.png')

In [ ]:
print(bls_results.transit_time)
print(bls_results.period[np.argmax(bls_results.power)])
print(bls_results.duration[np.argmax(bls_results.power)])
print(bls_results.transit_time[np.argmax(bls_results.power)])

In [ ]:
xxx

In [ ]:
def plot_results_(x_detrend_, y_detrend_, x_ref, x_bak_, y_bak_, smooth, m, bls_results):

    fig = plt.figure(figsize=(20,10))
    
# Plot the data
    plt.subplot(311)
    plt.plot(x_bak_, y_bak_, "k", label="raw data")
    plt.plot(x_detrend_, smooth, lw = 3., label = 'sav-gol filter')
    plt.plot(x_bak_[~m], y_bak_[~m], "xr", label="outliers")
    plt.legend(fontsize=12)
    plt.xlim(x_bak_.min(), x_bak_.max())
    plt.xlabel("time")
    plt.ylabel("flux")
        
    plt.subplot(312)
    plt.plot(x_detrend_, y_detrend_, "k", label="data")
    plt.legend(fontsize=12)
    plt.xlim(x_detrend_.min(), x_detrend_.max())
    plt.xlabel("time")
    plt.ylabel("flux")
    
    plt.subplot(121)
    
    index = np.argmax(bls_results.power)
    period = bls_power.period[index]
    t0 = bls_power.transit_time[index]
    depth = bls_power.depth[index]
    duration = bls_power.duration[index]
    
    plt.plot(np.log10(bls_results[0].period), bls_results[0].power, "k")
    plt.title('BLS periodogram')
    # Plot the folded transit
    plt.subplot(122)
    p = period[0]
    x_fold = (x_detrend_ - x_ref + 0.5*p) % p - 0.5*p
    m = np.abs(x_fold) < 0.5
    plt.plot(x_fold[m], y[m], ".k")    
    # Overplot the phase binned light curve
    bins = np.linspace(-0.5, 0.5, 32)
    denom, _ = np.histogram(x_fold, bins)
    num, _ = np.histogram(x_fold, bins, weights=y)
    denom[num == 0] = 1.0
    plt.plot(0.5*(bins[1:] + bins[:-1]), num / denom, color="C1")
    plt.title('TIC ' + str(tic) + "; P =" + str(np.round(p,2)) + "; t0 =" + str(round(x_ref,2)))
    plt.xlabel('Phase')
    
    return fig#x_detrend_, y_detrend_, x_ref, x_bak_, y_bak_, smooth, m

In [ ]:
#xx = plot_results_(time_detrend_, flux_detrend_, t0, x_bak_, y_bak_, smooth, m, bls_results)

In [ ]:
#fig = plt.figure(figsize=(20,5))
#plt.plot(time_detrend_, flux_detrend_, 'k-o')
#plt.xlim(0.,1)

In [ ]:
print(rs)

In [ ]:
#rEarth_to_rSun = 0.0091577
#rEarth_to_rJupiter = 0.08921
rJupiter_to_rSun = 0.10045

rp = np.asarray(rs)*(np.asarray(1e-3*depth)**0.5)/rJupiter_to_rSun#rEarth_to_rJupiter/rEarth_to_rSun

ss = rp.argsort()

#for kk in range(len(tic_bak)):
#    print(tic_bak[kk])
print('TIC, Per, R[RJup], Comment')
print()

for ii in range(len(tic_bak[ss])):
    print(tic_bak[ss[ii]], period[ss[ii]], np.round(rp[ss[ii]],2), comment[ss[ii]])

In [ ]:
xxx